In [2]:
#%% discover.py
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

df = pd.read_parquet('data/preprocess/derived.parquet')

In [3]:
df.head()

,streetAddress,suburb,state,postcode,propertyType,bed,bath,car,landSizeM2,floorSizeM2,...,avg_floor_size,price_future_6m,price_future_6m_delta,price_future_6m_diff,price_future_12m,price_future_12m_delta,price_future_12m_diff,price_future_12m_smooth,price_future_12m_smooth_delta,price_future_12m_smooth_diff
0,112 BRUNSWICK ROAD,MITCHAM,VIC,3132,UNIT,3.0,1.0,2.0,619.0,102.0,...,105.20,1052000.0,0.137297,127000.0,1052000.0,0.137297,127000.0,920791.666667,-0.004550,-4208.333333
1,12 CARWEEN AVENUE,MITCHAM,VIC,3132,HOUSE,3.0,1.0,2.0,833.0,130.0,...,107.75,1052000.0,0.077869,76000.0,1052000.0,0.077869,76000.0,932333.333333,-0.044740,-43666.666667
2,3/55 CARWEEN AVENUE,MITCHAM,VIC,3132,HOUSE,2.0,1.0,2.0,217.0,106.0,...,NaN,1090000.0,NaN,NaN,1090000.0,NaN,NaN,947750.000000,NaN,NaN
3,7 ROTHERWOOD AVENUE,MITCHAM,VIC,3132,HOUSE,4.0,3.0,2.0,470.0,181.0,...,NaN,1052000.0,NaN,NaN,1052000.0,NaN,NaN,960000.000000,NaN,NaN
4,6/35 HARRISON STREET,MITCHAM,VIC,3132,UNIT,3.0,2.0,2.0,286.0,142.0,...,126.75,1052000.0,0.076215,74500.0,1052000.0,0.076215,74500.0,971541.666667,-0.006095,-5958.333333


In [4]:
(df.groupby(['saleYearMonth', 'street', 'propertyType'])
   .agg({'salePrice': 'mean'})
   .assign(counts=lambda x: df.groupby(['saleYearMonth', 'street', 'propertyType']).size())
   .reset_index()
   .sort_values(by=['saleYearMonth','salePrice'], ascending=False)
   .head(10))

,saleYearMonth,street,propertyType,salePrice,counts
2531,202508,Reserve Avenue,HOUSE,1800000.0,1
2532,202508,Rotherwood Avenue,HOUSE,1727000.0,1
2522,202508,Gillies Street,HOUSE,1614000.0,1
2523,202508,Haddon Court,HOUSE,1440000.0,1
2525,202508,Hopetoun Street,UNKNOWN,1332000.0,1
2526,202508,Ian Crescent,UNIT,1250000.0,1
2515,202508,Blossom Street,HOUSE,1241000.0,1
2517,202508,Carween Avenue,HOUSE,1160000.0,2
2533,202508,Rupert Street,HOUSE,1144000.0,1
2519,202508,Churinga Avenue,UNIT,1105000.0,1


In [ ]:
# Filter out zero price sales for analysis
df_filtered = df[df['salePrice'] > 0].copy()

# Test for missing months at street and property type level
def find_missing_months_by_group(df, group_cols):
    """Find missing months for each group defined by group_cols"""
    results = []
    
    for group_values, group_df in df.groupby(group_cols):
        # Get all months for this group
        group_months = group_df['saleYearMonth'].unique()
        min_month = group_months.min()
        max_month = group_months.max()
        
        # Create complete range of months for this group's timespan
        expected_months = []
        current = min_month
        while current <= max_month:
            expected_months.append(current)
            # Increment month
            year = current // 100
            month = current % 100
            if month == 12:
                current = (year + 1) * 100 + 1
            else:
                current = current + 1
        
        # Find missing months
        missing_months = set(expected_months) - set(group_months)
        
        if missing_months:
            group_dict = dict(zip(group_cols, group_values if len(group_cols) > 1 else [group_values]))
            group_dict.update({
                'missing_months': sorted(missing_months),
                'missing_count': len(missing_months),
                'total_months_span': len(expected_months),
                'months_with_data': len(group_months),
                'min_month': min_month,
                'max_month': max_month
            })
            results.append(group_dict)
    
    return pd.DataFrame(results)

group_cols = ['suburb', 'propertyType', 'bed_bucket', 'bath_bucket', 'floor_bucket']
# Test for missing months by street and property type
missing_by_street_type = find_missing_months_by_group(df_filtered, )

print(f"Groups with missing months: {len(missing_by_street_type)}")
if len(missing_by_street_type) > 0:
    print(f"Total unique street-propertyType combinations: {df_filtered.groupby(['street', 'propertyType']).ngroups}")
    print(f"Percentage with missing months: {len(missing_by_street_type) / df_filtered.groupby(['street', 'propertyType']).ngroups * 100:.1f}%")
    print("\nSample of groups with missing months:")
    print(missing_by_street_type.head())
else:
    print("No missing months found at street-propertyType level")

Groups with missing months: 283
Total unique street-propertyType combinations: 374
Percentage with missing months: 75.7%

Sample of groups with missing months:
             street propertyType  \
0       Agra Street        HOUSE   
1     Albert Street        HOUSE   
2     Albert Street         UNIT   
3  Alexander Street        HOUSE   
4       Alpha Court        HOUSE   

                                      missing_months  missing_count  \
0  [201902, 201903, 201904, 201906, 201907, 20190...             63   
1  [201706, 201707, 201708, 201709, 201710, 20171...             88   
2  [201904, 201905, 201906, 201907, 201908, 20190...             56   
3  [202209, 202211, 202212, 202301, 202302, 20230...             22   
4  [202309, 202310, 202311, 202312, 202401, 20240...             16   

   total_months_span  months_with_data  min_month  max_month  
0                 68                 5     201901     202408  
1                 97                 9     201705     202505  
2      

In [1]:
df.groupby(['saleYearMonth', 'street', 'propertyType']).count()['salePrice']

NameError: name 'df' is not defined

In [ ]:

# find all missing months in the dataset
all_months = df_filtered['saleYearMonth'].unique()
min_month = all_months.min()
max_month = all_months.max()

# Create a complete range of months
expected_months = []
current = min_month
while current <= max_month:
    expected_months.append(current)
    # Increment month
    year = current // 100
    month = current % 100
    if month == 12:
        current = (year + 1) * 100 + 1
    else:
        current = current + 1

# Find missing months
missing_months = set(expected_months) - set(all_months)
if missing_months:
    print(f"Missing months: {sorted(missing_months)}")
else:
    print("No missing months found")

print(f"Data spans from {min_month} to {max_month}")
print(f"Total months in range: {len(expected_months)}")
print(f"Months with data: {len(all_months)}")
